In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import cv2
import numpy
from sklearn.preprocessing import LabelEncoder

In [2]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [3]:
class LeafDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [5]:
def load_images_and_labels(directory):
    images = []
    labels = []
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            image = cv2.imread(file_path)

            if image is None:
                continue

            image = cv2.resize(image, (128, 128))
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            images.append(gray_image)
            labels.append(class_folder)
    return numpy.array(images), numpy.array(labels)

In [6]:
train_images, train_labels = load_images_and_labels('../../dataset/train')
test_images, test_labels = load_images_and_labels('../../dataset/test')

train_images_tensor = [transform(img) for img in train_images]
test_images_tensor = [transform(img) for img in test_images]

In [7]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [8]:
train_dataset = LeafDataset(train_images_tensor, train_labels_encoded, transform=None)
test_dataset = LeafDataset(test_images_tensor, test_labels_encoded, transform=None)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [10]:
num_classes = len(label_encoder.classes_)
model = CNNModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete.")

Epoch [1/10], Loss: 1.8379
Epoch [2/10], Loss: 1.3818
Epoch [3/10], Loss: 1.1912
Epoch [4/10], Loss: 1.0614
Epoch [5/10], Loss: 0.9373
Epoch [6/10], Loss: 0.8468
Epoch [7/10], Loss: 0.7647
Epoch [8/10], Loss: 0.6913
Epoch [9/10], Loss: 0.6610
Epoch [10/10], Loss: 0.6007
Training complete.


In [12]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'CNN Model Test Accuracy: {accuracy:.2f}%')

CNN Model Test Accuracy: 77.83%
